# IMPORTS

In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
# from sklearn.metrics import confusion_matrix, precision_score, recall_score

# UNPICKLE

In [ ]:
def unpickle_it(name):
    filepath = '/Users/gandalf/Documents/data/'+name+'.pkl'
    return pd.read_pickle(filepath)

In [ ]:
user_df = unpickle_it('user_df')
# unpickle_it(message_df,'message_df')
convo_df = unpickle_it('convo_df')
# unpickle_it(lastmessage_df,'lastmessage_df')

In [ ]:
user_df.head(2)

In [ ]:
convo_df.head(2)

In [ ]:
print("Make sure we're not useing ECT data")
print(pd.to_datetime(convo_df.timestamp.max()*1000000))

In [ ]:
print("Messages with responses:    {}".format((convo_df.response == True).sum()))
print("Messages without responses: {}".format((convo_df.response == False).sum()))
print("Total messages:             {}".format(len(convo_df)))

# GET X AND y

In [ ]:
simple_cols = ['response','first_uid','second_uid']
simple_df = convo_df[simple_cols]

ad,sg,sr,sc,sn,ss,t = [],[],[],[],[],[],[]
c=0
for index, row in simple_df.iterrows():
    try:
        first = user_df.loc[str(row.first_uid)]
        second = user_df.loc[str(row.second_uid)]
        ad.append(abs(first.age - second.age))
        sg.append(first.gender == second.gender)
        sr.append(first.inRelationship == second.inRelationship)
        sc.append(first.isClean == second.isClean)
        sn.append(first.isNight == second.isNight)
        ss.append(first.isStudent == second.isStudent)
        t.append(False)
    except:
        ad.append('trouble')
        sg.append('trouble')
        sr.append('trouble')
        sc.append('trouble')
        sn.append('trouble')
        ss.append('trouble')
        t.append(True)
        
        c += 1

simple_df['age_dif'] = ad
simple_df['same_gender'] = sg
simple_df['same_relate'] = sr
simple_df['same_clean'] = sc
simple_df['same_night'] = sn
simple_df['same_student'] = ss
simple_df['trouble'] = t

simple_df = simple_df.drop(simple_df[simple_df.trouble].index)
simple_df = simple_df.drop(['trouble'], axis=1)
print("Num bad rows: {}".format(c))
simple_df.head(2)

In [ ]:
X = simple_df
y = X.response.map({True:1,False:0})
X = X.drop(['first_uid','second_uid','response'], axis=1)
print(len(X))
print(len(y))#.sum()

# BASELINE RANDOM FOREST

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test.values))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

# UNDERSAMPLE

In [ ]:
def div_count_pos_neg(X, y):
    """Helper function to divide X & y into positive and negative classes
    and counts up the number in each.
    Parameters
    ----------
    X : ndarray - 2D
    y : ndarray - 1D
    Returns
    -------
    negative_count : Int
    positive_count : Int
    X_positives    : ndarray - 2D
    X_negatives    : ndarray - 2D
    y_positives    : ndarray - 1D
    y_negatives    : ndarray - 1D
    """
    negatives, positives = y == 0, y == 1
    negative_count, positive_count = np.sum(negatives), np.sum(positives)
    X_positives, y_positives = X[positives], y[positives]
    X_negatives, y_negatives = X[negatives], y[negatives]
    return negative_count, positive_count, X_positives, \
           X_negatives, y_positives, y_negatives

In [ ]:
def undersample(X, y, tp):
    """Randomly discards negative observations from X & y to achieve the
    target proportion of positive to negative observations.

    Parameters
    ----------
    X  : ndarray - 2D
    y  : ndarray - 1D
    tp : float - range [0, 1], target proportion of positive class observations

    Returns
    -------
    X_undersampled : ndarray - 2D
    y_undersampled : ndarray - 1D
    """
    if tp < np.mean(y):
        return X, y
    neg_count, pos_count, X_pos, X_neg, y_pos, y_neg = div_count_pos_neg(X, y)
    negative_sample_rate = (pos_count * (1 - tp)) / (neg_count * tp)
    negative_keepers = np.random.choice(a=[False, True], size=neg_count,
                                        p=[1 - negative_sample_rate,
                                           negative_sample_rate])
    X_negative_undersampled = X_neg[negative_keepers]
    y_negative_undersampled = y_neg[negative_keepers]
    X_undersampled = np.vstack((X_negative_undersampled, X_pos))
    y_undersampled = np.concatenate((y_negative_undersampled, y_pos))

    return X_undersampled, y_undersampled

In [ ]:
X_und, y_und = undersample(X, y, .5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_und, y_und, random_state=17)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

# OVER SAMPLE

In [ ]:
def oversample(X, y, tp):
    """Randomly choose positive observations from X & y, with replacement
    to achieve the target proportion of positive to negative observations.

    Parameters
    ----------
    X  : ndarray - 2D
    y  : ndarray - 1D
    tp : float - range [0, 1], target proportion of positive class observations

    Returns
    -------
    X_undersampled : ndarray - 2D
    y_undersampled : ndarray - 1D
    """
    if tp < np.mean(y):
        return X, y
    neg_count, pos_count, X_pos, X_neg, y_pos, y_neg = div_count_pos_neg(X, y)
    positive_range = np.arange(pos_count)
    positive_size = (tp * neg_count) / (1 - tp)
    positive_idxs = np.random.choice(a=positive_range,
                                     size=int(positive_size),
                                     replace=True)
    X_positive_oversampled = X_pos[positive_idxs]
    y_positive_oversampled = y_pos[positive_idxs]
    X_oversampled = np.vstack((X_positive_oversampled, X_neg))
    y_oversampled = np.concatenate((y_positive_oversampled, y_neg))

    return X_oversampled, y_oversampled

In [ ]:
X_ove, y_ove = undersample(X, y, .5)

X_train, X_test, y_train, y_test = train_test_split(X_ove, y_ove, random_state=17)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))